# Nuevo topic model

In [56]:
import os
import pymongo
import re
import random
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

import spacy
from spacy.lang.en import English
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords


import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.models.coherencemodel import CoherenceModel

import pickle

import pyLDAvis
import pyLDAvis.gensim

nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['https', 'co', "amp", "lt", "gt"])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
#conexion mongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["twitter-memoria"]

csv_all = mydb["csv_all"]

In [58]:
string = '\n'

hilos_ref = []
all_hilos = []
tweets = csv_all.find({})

threads1 = {}
threads2 = {}
threads3 = {}
threads4 = {}
threads5 = {}
kthreads1 = {}
kthreads2 = {}
kthreads3 = {}
kthreads4 = {}
kthreads5 = {}
hilo_Tthreads = {}

for tweet in tweets:
    if tweet["hilo_ref"] not in hilos_ref:
        hilos_ref.append(tweet["hilo_ref"])

for hilo in hilos_ref:
    tweets_hilo = csv_all.find({"hilo_ref": hilo})
    lista_aux_hilos = []
    csv = ""
    for tweet_aux in tweets_hilo:
        lista_aux_hilos.append(tweet_aux["text"])
        csv = tweet_aux["csv"]
        hilo_csv = tweet_aux["hilo"]
    
    if csv == "csv1":
        threads1[hilo_csv] = lista_aux_hilos
        kthreads1[hilo_csv] = string.join(lista_aux_hilos)
    elif csv == "csv2":
        threads2[hilo_csv] = lista_aux_hilos
        kthreads2[hilo_csv] = string.join(lista_aux_hilos)
    elif csv == "csv3":
        threads3[hilo_csv] = lista_aux_hilos
        kthreads3[hilo_csv] = string.join(lista_aux_hilos)
    elif csv == "csv4":
        threads4[hilo_csv] = lista_aux_hilos
        kthreads4[hilo_csv] = string.join(lista_aux_hilos)
    elif csv == "csv5":
        threads5[hilo_csv] = lista_aux_hilos
        kthreads5[hilo_csv] = string.join(lista_aux_hilos)
        
    hilo_Tthreads[hilo_csv] = string.join(lista_aux_hilos)
        
Tthreads1 = list(kthreads1.values())
Tthreads2 = list(kthreads2.values())
Tthreads3 = list(kthreads3.values())
Tthreads4 = list(kthreads4.values())
Tthreads5 = list(kthreads5.values())


In [59]:
data = Tthreads1+Tthreads2+Tthreads3+Tthreads4+Tthreads5

data = [re.sub(r"@\S+", "", sent) for sent in data]

data = [re.sub(r"http\S+", "", sent) for sent in data]

data = [re.sub(r"#\S+", "", sent) for sent in data]

data = [" ".join(sent.split()) for sent in data]


In [60]:
data[1]

'(4) With censuring the Social Media and with controlling foreign countries through tech. (3) This opeation "LIVE" by Q, in China, had something to do with him - but was not his capture. (2) In Clowns Operations, and narrative drops. (5) We had this "waldo" moment, in Hong Kong. (6) Well, what the future holds for him? Doesn\'t look too good. /END (1) A thread about Snow White, and his many influences: FIRST, in the Q comms.'

In [61]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [62]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [64]:
# See trigram example
for datito in data_words:
    print(trigram_mod[bigram_mod[datito]])

['create', 'path', 'to', 'green', 'card', 'for', 'investors', 'include', 'any', 'children', 'brought', 'here', 'before', 'age', 'let', 'really', 'make', 'year', 'law', 'provisional', 'green', 'cards', 'direct', 'dhs', 'to', 'allow', 'people', 'to', 'take', 'the', 'steps', 'to', 'correct', 'their', 'immigration', 'status', 'penalties', 'besides', 'deportation', 'vote', 'out', 'elected', 'officials', 'with', 'close', 'ties', 'to', 'nativist', 'white', 'nationalist', 'or', 'fearmongering', 'groups', 'such', 'as', 'fair', 'cis', 'numbersusa', 'us', 'inc', 'kkk', 'vdare', 'etc', 'give', 'immig', 'judges', 'true', 'independence', 'more', 'support', 'staff', 'backlog', 'relief', 'for', 'india', 'china', 'philippines', 'mexico', 'expand', 'esl', 'instruction', 'create', 'state', 'offices', 'of', 'new', 'americans', 'make', 'real', 'use', 'of', 'visas', 'to', 'take', 'down', 'cartels', 'make', 'filing', 'for', 'citizenship', 'free', 'or', 'almost', 'free', 'create', 'path', 'to', 'green', 'card

['extraordinary', 'evidence', 'at', 'treasury', 'committee', 'from', 'jon', 'thompson', 'ceo', 'of', 'hmrc', 'on', 'customs', 'and', 'brexit', 'today', 'the', 'brexiter', 'favourite', 'max', 'fac', 'would', 'cost', 'business', 'between', 'and', 'bn', 'year', 'that', 'almost', 'of', 'gdp', 'just', 'for', 'filling', 'in', 'forms', 'thanks', 'we', 'think', 'we', 'can', 'manage', 'the', 'risk', 'we', 'think', 'we', 'can', 'he', 'said', 'he', 'didn', 'sound', 'so', 'sure', 'and', 'the', 'potential', 'backdoor', 'risk', 'applies', 'to', 'both', 'models', 'he', 'added', 'didn', 'sound', 'like', 'officials', 'think', 'either', 'is', 'sensible', 'mr', 'thompson', 'said', 'he', 'did', 'not', 'expect', 'the', 'eu', 'to', 'reciprocate', 'over', 'the', 'customs', 'partnership', 'what', 'that', 'means', 'is', 'uk', 'collects', 'tariffs', 'for', 'eu', 'and', 'hands', 'it', 'over', 'when', 'ship', 'lands', 'in', 'felixtowe', 'and', 'drives', 'to', 'calais', 'but', 'if', 'ship', 'first', 'lands', 'in',

['now', 'we', 'fly', 'over', 'to', 'ny', 'where', 'there', 'is', 'similar', 'site', 'punch', 'both', 'feins', 'using', 'dashes', 'after', 'first', 'numbers', 'here', 'foreign', 'governments', 'are', 'not', 'the', 'only', 'ones', 'looking', 'closely', 'at', 'public', 'filings', 'as', 'and', 'dig', 'in', 'then', 'look', 'carefully', 'amended', 'bylaws', 'filed', 'almost', 'four', 'years', 'late', 'on', 'june', 'in', 'ny', 'clinton', 'family', 'illegally', 'gutted', 'public', 'charity', 'status', 'on', 'november', 'as', 'they', 'duped', 'and', 'incoming', 'directors', 'then', 'hid', 'this', 'massive', 'error', 'past', 'the', 'election', 'in', 'ny', 'ny', 'laws', 'also', 'require', 'specifics', 'concerning', 'all', 'government', 'grants', 'why', 'are', 'there', 'still', 'no', 'amended', 'disclosures', 'from', 'onwards', 'especially', 'concerning', 'foreign', 'government', 'grants', 'oh', 'wait', 'because', 'is', 'hack', 'in', 'thrall', 'to', 'the', 'clintons', 'please', 'note', 'that', 'wa

['the', 'real', 'issue', 'is', 'if', 'the', 'was', 'doing', 'these', 'behavior', 'and', 'drug', 'experiments', 'in', 'the', 'makes', 'us', 'think', 'they', 'just', 'decided', 'to', 'stop', 'they', 'think', 'we', 'are', 'sheep', 'they', 'are', 'wrong', 'gave', 'us', 'back', 'our', 'voice', 'stated', 'by', 'senator', 'kennedy', 'there', 'are', 'perhaps', 'any', 'number', 'of', 'americans', 'who', 'are', 'walking', 'around', 'today', 'who', 'were', 'given', 'drugs', 'with', 'all', 'the', 'kinds', 'of', 'physical', 'and', 'psychological', 'damage', 'than', 'can', 'be', 'caused', 'page', 'part', 'of', 'that', 'led', 'to', 'its', 'exposure', 'was', 'the', 'death', 'of', 'frank', 'olsen', 'who', 'was', 'drugged', 'with', 'lsd', 'by', 'the', 'cia', 'without', 'his', 'consent', 'in', 'the', 'one', 'week', 'later', 'he', 'threw', 'himself', 'off', 'balcony', 'or', 'did', 'he', 'can', 'someone', 'take', 'gander', 'on', 'wtf', 'means', 'in', 'exchange', 'for', 'this', 'donation', 'the', 'was', 'gr

['an', 'anon', 'responding', 'to', 'question', 'who', 'performs', 'in', 'circus', 'thought', 'the', 'answer', 'was', 'clowns', 'mentioned', 'snowden', 'girlfriend', 'asks', 'who', 'the', 'father', 'is', 'his', 'name', 'is', 'jonathan', 'mills', 'need', 'to', 'dig', 'into', 'his', 'background', 'appears', 'to', 'want', 'us', 'to', 'push', 'this', 'hash', 'tag', 'responds', 'to', 'an', 'anon', 'who', 'had', 'concerns', 'about', 'ibor', 'clarifies', 'st', 'amendment', 'can', 'be', 'enforced', 'upon', 'private_companies', 'yes', 'they', 'risk', 'losing', 'cash', 'but', 'clowns', 'make', 'up', 'for', 'it', 'our', 'tax', 'payer', 'dollars', 'there', 'must', 'be', 'way', 'to', 'prove', 'that', 'if', 'we', 'are', 'funding', 'we', 'should', 'control', 'just', 'theory', 'here', 'but', 'if', 'snowden', 'has', 'flipped', 'because', 'he', 'is', 'now', 'according', 'to', 'liability', 'to', 'clowns', 'like', 'jp', 'barlow', 'who', 'was', 'ed', 'became', 'is', 'team', 'china', 'assistance', 'helping',

...

['robert_mercer', 'the', 'investor', 'in', 'he', 'studied', 'at', 'university', 'of', 'illinois', 'and', 'was', 'co', 'ceo', 'of', 'renaissance', 'technologies', 'an', 'old', 'us', 'hedge', 'fund', 'company', 'also', 'studied', 'amp', 'from', 'illinois', 'and', 'when', 'the', 'stock', 'mkt', 'shoot', 'up', 'suddenly', 'pull', 'out', 'all', 'investment', 'which', 'will', 'crash', 'indian', 'stock_market', 'was', 'this', 'planned', 'by', 'schedule', 'time', 'was', 'june', 'july', 'but', 'seems', 'it', 'is', 'now', 'postponed', 'after', 'october', 'and', 'before', 'december', 'my', 'guess', 'would', 'be', 'diwali', 'alexander_nix', 'met', 'and', 'others', 'during', 'india', 'visit', 'nix', 'is', 'well', 'connected', 'with', 'robert_mercer', 'the', 'investor', 'in', 'and', 'ca', 'is', 'scl', 'group', 'company', 'robert', 'is', 'known', 'well', 'in', 'political', 'circle', 'and', 'the', 'bomb', 'son', 'salil', 'pitroda', 'worked', 'with', 'facebook', 'salil', 'son', 'of', 'who', 'is', 'orga

['ninjago', 'is', 'mixed', 'bag', 'but', 'not', 'gonna', 'say', 'this', 'isn', 'awesome', 'she', 'conked', 'out', 'hours', 'ago', 'but', 'this', 'is', 'how', 'think', 'she', 'rank', 'today', 'attractions', 'safari', 'trek', 'merry', 'go', 'round', 'miniland', 'new', 'orleans', 'playing', 'with', 'some', 'legos', 'fairy', 'tale', 'brook', 'meeting', 'nya', 'electric', 'hand', 'dryer', 'in', 'bathroom', 'tie', 'punted', 'on', 'benihana', 'she', 'too', 'tired', 'still', 'managed', 'to', 'rustle', 'up', 'dece', 'rite', 'of', 'passage', 'for', 'her', 'she', 'insisted', 'take', 'this', 'pic', 'weird', 'ugh', 'she', 'spilled', 'chocolate', 'milk', 'on', 'herself', 'we', 'cleaned', 'it', 'before', 'it', 'could', 'stain', 'but', 'we', 'have', 'less', 'total', 'chocolate', 'milk', 'now', 'we', 've', 'agreed', 'she', 'gets', 'two', 'small', 'lego', 'sets', 'on', 'this', 'trip', 'she', 'turns', 'five', 'later', 'this', 'month', 'her', 'first', 'selection', 'was', 'from', 'the', 'lego', 'friends', 

['to', 'my', 'fellow', 'healthcare', 'providers', 'remember', 'pain', 'is', 'an', 'important', 'vital', 'sign', 'that', 'should', 'always', 'be', 'documented', 'background', 'info', 'on', 'bruce', 'and', 'nellie', 'orh', 'and', 'their', 'ham_radio_license', 'another', 'anon', 'made', 'this', 'observation', 'today', 'due', 'to', 'speculation', 'that', 'is', 'not', 'the', 'original', 'who', 'posted', 'in', 'october', 'and', 'november', 'he', 'posted', 'two', 'pictures', 'that', 'seem', 'to', 'be', 'taken', 'just', 'seconds', 'apart', 'the', 'one', 'on', 'the', 'left', 'is', 'the', 'original', 'image', 'posted', 'in', 'november', 'the', 'one', 'on', 'the', 'right', 'has', 'the', 'filename', 'a_', 'png', 'info', 'on', 'ed', 'callaghan', 'trust', 'callaghan', 'asked', 'if', 'we', 've_seen', 'the', 'interview', 'with', 'paul', 'ryan', 'assume', 'he', 'speaking', 'of', 'this', 'one', 'where', 'he', 'admits', 'that', 'president', 'trump', 'request', 'to', 'have', 'the', 'justice_department', '

['the', 'first', 'year', 'lived', 'out', 'in', 'the', 'country', 'realized', 'very', 'quickly', 'that', 'even', 'though', 'isolated', 'there', 'can', 'still', 'be', 'danger', 'and', 'it', 'up', 'to', 'me', 'to', 'protect', 'my', 'family', 'from', 'that', 'danger', 'replied', 'simply', 'no', 'and', 'quickly', 'drove', 'away', 'to', 'get', 'home', 'an', 'call', 'the', 'authorities', 'he', 'wasn', 'going', 'to', 'die', 'of', 'starvation', 'before', 'proper', 'people', 'came', 'to', 'help', 'him', 'living', 'out', 'in', 'the', 'middle', 'of', 'nowhere', 'doesn', 'mean', 'one', 'is', 'completely', 'safe', 'though', 'there', 'these', 'things', 'called', 'roads', 'people', 'use', 'them', 'no', 'cold', 'hearted', 'asshole', 'don', 'mind', 'helping', 'people', 'but', 'after', 'years', 'of', 'living', 'in', 'the', 'city', 've', 'learned', 'to', 'stay', 'away', 'from', 'the', 'drug', 'users', 'especially', 'when', 'they', 're', 'tripping', 'out', 'they', 're', 'dangerous', 'period', 'to', 'themse

['she', 'could', 'not', 'identify', 'the', 'groom', 'he', 'wasn', 'my_dad', 'dad', 'and', 'he', 'wasn', 'my', 'grandma', 'current', 'husband', 'very', 'polite', 'heavily', 'tattooed', 'latvian', 'man', 'missing', 'thumb', 'named', 'janus', 'marion', 'my', 'mother', 'said', 'who', 'are', 'you', 'marrying', 'in', 'this', 'photo', 'the', 'reason', 'papa', 'john', 'got', 'really', 'twitchy', 'in', 'well', 'that', 'was', 'the', 'year', 'canada', 'announced', 'it', 'was', 'open', 'to', 'stripping', 'citizenship', 'from', 'and', 'deporting', 'war', 'criminals', 'following', 'the', 'recommendations', 'of', 'deschenes', 'commission', 'my_dad', 'drove', 'to', 'toronto', 'and', 'developed', 'relationship', 'with', 'his', 'dad', 'we', 'tagged', 'along', 'sometimes', 'and', 'brought', 'him', 'cartons', 'of', 'cigarettes', 'and', 'chocolate', 'bars', 'etc', 'when', 'she', 'got', 'back', 'her', 'parents', 'said', 'that', 'it', 'would', 'be', 'best', 'to', 'just', 'put', 'all', 'that', 'unpleasantness

In [65]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [66]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [67]:
for datlem in data_lemmatized:
    print(datlem)

['create', 'path', 'green', 'card', 'investor', 'include', 'child', 'bring', 'age', 'let', 'really', 'make', 'year', 'law', 'provisional', 'green', 'card', 'direct', 'dhs', 'allow', 'people', 'take', 'step', 'correct', 'immigration', 'status', 'penalty', 'deportation', 'vote', 'elect', 'official', 'close', 'tie', 'nativist', 'white', 'nationalist', 'fearmonger', 'group', 'fair', 'cis', 'numbersusa', 'inc', 'kkk', 'vdare', 'give', 'immig', 'judge', 'true', 'independence', 'support', 'staff', 'backlog', 'relief', 'india', 'china', 'philippine', 'mexico', 'expand', 'esl', 'instruction', 'create', 'state', 'office', 'new', 'american', 'make', 'real', 'use', 'visa', 'take', 'cartel', 'make', 'file', 'citizenship', 'free', 'almost', 'free', 'create', 'path', 'green', 'card', 'longterm', 'tps', 'holder', 'recognize', 'deporte', 'group', 'asylum', 'much', 'do', 'stick', 'border_wall', 'daca', 'gutting', 'protection', 'currently', 'undocumented', 'gutt', 'decade', 'work', 'pay', 'someone', 'reb

['fly', 'similar', 'site', 'punch', 'fein', 'use', 'dash', 'first', 'number', 'foreign', 'government', 'one', 'look', 'closely', 'public', 'filing', 'dig', 'look', 'carefully', 'amend', 'bylaw', 'file', 'almost', 'year', 'late', 'june', 'clinton', 'family', 'illegally', 'gutt', 'public', 'charity', 'status', 'november', 'dupe', 'incoming', 'director', 'hide', 'massive', 'error', 'election', 'law', 'also', 'require', 'specific', 'concern', 'government', 'grant', 'still', 'amend', 'disclosure', 'onwards', 'especially', 'concern', 'foreign', 'government', 'grant', 'wait', 'hack', 'thrall', 'clinton', 'note', 'california', 'long', 'state', 'investigate', 'clinton', 'entity', 'run', 'roughshod', 'willinmgly', 'dupe', 'donor', 'high', 'income', 'tax', 'rate', 'state', 'dficit', 'must', 'step', 'game', 'fraud', 'main', 'question', 'many', 'other', 'miss', 'obvious', 'fraud', 'long', 'ever', 'february', 'key', 'figure', 'tell', 'tweet', 'provide', 'link', 'november', 'agreement', 'national', '

['reveal', 'woman', 'care', 'florida', 'shooter', 'file', 'court', 'paper', 'demand', 'control', 'inheritance', 'mother', 'day', 'massacre', 'friday', 'shoot', 'half', 'brother', 'commit', 'cia', 'use', 'jounalist', 'clergy', 'intelligence', 'operation', 'full', 'doc', 'obama', 'implicate', 'german', 'journalist', 'udo', 'ulfkotte', 'murder', 'blow', 'whistle', 'cia', 'control', 'medium', 'never', 'end', 'fauntroy', 'know', 'student', 'activist', 'plan', 'ad', 'proof', 'rothschild', 'cia', 'fund', 'heart', 'attack', 'deadly', 'jeremiah', 'leary']
['anon', 'respond', 'question', 'perform', 'circus', 'thought', 'answer', 'clown', 'mention', 'snowden', 'girlfriend', 'ask', 'father', 'name', 'jonathan', 'mill', 'need', 'dig', 'background', 'appear', 'want', 'push', 'hash', 'tag', 'respond', 'anon', 'concern', 'ibor', 'clarifie', 'amendment', 'enforce', 'private', 'company', 'risk', 'lose', 'cash', 'clown', 'make', 'tax', 'payer', 'dollar', 'must', 'way', 'prove', 'funding', 'control', 'the

['product', 'dissonance', 'cabal', 'want', 'look', 'wizard', 'convince', 'answer', 'far', 'complicated', 'solve', 'urge', 'tune', 'tomorrow', 'nothing', 'news', 'traffic', 'weather', 'sport', 'give', 'validity', 'matter', 'expert', 'choose', 'attack', 'feel', 'shame', 'score', 'point', 'get', 'hit', 'dopamine', 'dark', 'magic', 'checkerboard', 'confusion', 'light', 'shadow', 'segment', 'end', 'unresolved', 'anatomy', 'msm', 'segment', 'present', 'extreme', 'topic', 'pander', 'difference', 'hire', 'expert', 'compete', 'camera', 'time', 'end', 'segment', 'unresolv', 'complicated', 'discern', 'right', 'answer', 'let', 'analyse', 'chemically', 'find', 'element', 'msm', 'broadcast', 'someone', 'complain', 'dishonesty', 'look', 'back', 'formula', 'bet', 'follow', 'perfectly', 'goal', 'make', 'feel', 'helpless', 'pumping', 'adrenlin', 'snake', 'charming', 'set', 'actor', 'pretend', 'expert', 'feel', 'adrenalin', 'helplessness', 'time', 'happen', 'neuro', 'pathway', 'remember', 'passion', 'che

['next', 'step', 'collective', 'work', 'want', 'endure', 'real', 'change', 'child', 'generation', 'come', 'must', 'take', 'piece', 'work', 'time', 'institutional', 'patriarchy', 'write', 'statute', 'next', 'challenge', 'current', 'woman', 'movement', 'reform', 'law', 'many', 'convict', 'rapist', 'custody', 'visitation', 'child', 'result', 'rape', 'domestic', 'child', 'abuser', 'custody', 'change', 'law', 'must', 'woman', 'congress', 'lobby', 'meet', 'ask', 'make', 'law', 'consistent', 'human', 'civil_right', 'woman', 'child', 'legal', 'property', 'man', 'sexual', 'relation', 'irrespective', 'consensual', 'reason', 'story', 'widespread', 'prevelent', 'thank', 'large', 'part', 'law', 'still', 'book', 'allow', 'sexual', 'predation', 'child', 'young', 'woman', 'accountability', 'undergirding', 'law', 'concept', 'female', 'less', 'male', 'current', 'child', 'rape', 'law', 'state', 'state', 'contact', 'state', 'legislator', 'move', 'st_century', 'dark', 'age', 'archaic', 'idea', 'child', 'wo

...

['lucky', 'deal', 'loss', 'right', 'story', 'end', 'place', 'special', 'register', 'citizen', 'landlord', 'bank', 'employer', 'require', 'check', 'status', 'special', 'number', 'sum', 'look', 'large', 'group', 'people', 'people', 'contribute', 'country', 'many', 'decade', 'suddenly', 'ask', 'prove', 'status', 'environment', 'hostile', 'onus', 'person', 'investigation', 'citizen', 'want', 'rent', 'flat', 'person', 'also', 'want', 'citizen', 'citizen', 'give', 'landlord', 'special', 'number', 'check', 'special', 'register', 'prior', 'agreement', 'landlord', 'choose', 'else', 'equal', 'wide', 'context', 'course', 'strategy', 'fact', 'reach', 'continuously', 'extend', 'traced', 'tip', 'iceberg', 'evidence', 'overwhelming', 'lot', 'actively', 'create', 'next', 'windrush', 'generation', 'stop', 'never', 'forget', 'fond', 'human_right', 'act', 'implication', 'briton', 'government', 'want', 'replace', 'universal', 'human_right', 'act', 'authority', 'know', 'anything', 'much', 'status', 'secure

['memo', 'est', 'resume', 'fait', 'alternatifs', 'entretenus', 'par', 'et', 'alt', 'right', 'defense', 'sont', 'rendus', 'ici', 'honorable', 'mai', 'enjeu', 'est', 'pas', 'dan', 'partisan', 'qui', 'sont', 'convaincus', 'la', 'conspiration', 'du', 'systeme', 'contre', 'lui', 'podcast', 'mon', 'passage', 'memo', 'et', 'dan', 'journal', 'mathieu', 'laurent', 'sur', 'durant', 'soiree', 'infos', 'sont', 'venus', 'discrediter', 'memo', 'qui', 'pourrait', 'transformer', 'boomerang', 'pour', 'part', 'fox', 'meme', 'nune', 'admis', 'avoir', 'vu', 'lui', 'meme', 'demand', 'autorisation', 'ecoute', 'qu_il', 'critique', 'autre', 'part', 'cour', 'fisa', 'qui', 'delivra', 'autorisation', 'ecoute', 'sur', 'conseiller', 'source', 'du', 'fbi', 'etait', 'enquete', 'politique', 'opposition', 'car', 'fbi', 'avait', 'informe', 'tout', 'savoir', 'si', 'opinion', 'majoritairement', 'accrochera', 'memo', 'et', 'acceptera', 'decision', 'fondera', 'dessus', 'probable', 'renvoi', 'rosenstein', 'puis', 'procureur

['robert_mercer', 'investor', 'study', 'university', 'illinois', 'ceo', 'renaissance', 'technology', 'old', 'hedge', 'fund', 'company', 'also', 'study', 'illinois', 'stock', 'mkt', 'shoot', 'suddenly', 'pull', 'investment', 'crash', 'indian', 'stock_market', 'plan', 'schedule', 'time', 'june', 'seem', 'postpone', 'october', 'december', 'guess', 'would', 'diwali', 'meet', 'other', 'india', 'visit', 'nix', 'well', 'connected', 'robert_mercer', 'investor', 'ca', 'scl', 'group', 'company', 'robert', 'know', 'well', 'political', 'circle', 'bomb', 'son', 'salil', 'pitroda', 'work', 'facebook', 'salil', 'son', 'organize', 'comedy', 'talk', 'show', 'world', 'work', 'hedge', 'fund', 'unit', 'citigroup', 'manhattan', 'hedge', 'fund', 'king', 'hedge', 'fund', 'robert_mercer', 'well', 'know', 'nix', 'seed', 'task', 'invest', 'indian', 'psu', 'big', 'company', 'shell', 'company', 'use', 'note', 'note', 'difficult', 'track', 'investment', 'size', 'cr', 'sir', 'explain', 'note', 'better', 'may', 'dis

['fellow', 'healthcare', 'provider', 'remember', 'pain', 'important', 'vital', 'sign', 'always', 'document', 'background', 'info', 'bruce', 'nellie', 'orh', 'ham_radio', 'license', 'anon', 'make', 'observation', 'today', 'due', 'speculation', 'original', 'post', 'october', 'november', 'post', 'picture', 'seem', 'take', 'second', 'apart', 'leave', 'original', 'image', 'post', 'november', 'right', 'filename', '_', 'png', 'info', 'callaghan', 'trust', 'callaghan', 'ask', 'see', 'interview', 'paul', 'ryan', 'assume', 'speak', 'admit', 'president', 'trump', 'request', 'justice_department', 'look', 'politicization', 'warrant', 'future_prov', 'past', 'november', 'potus', 'visit', 'asia', 'post', 'image', 'note', 'filename', 'metadata', '_', 'png', 'kib', 'implication', 'travel', 'access', 'president', 'air', 'force', 'link', 'respond', 'speculation', 'compromise', 'subvert', 'deep_state', 'current', 'post', 'october', 'november', 'last', 'year', 'thread', 'hopefully', 'put', 'fear', 'rest', '

['could', 'identify', 'groom', 'dad', 'dad', 'grandma', 'current', 'husband', 'polite', 'heavily', 'tattoo', 'latvian', 'man', 'miss', 'thumb', 'name', 'janus', 'marion', 'mother', 'say', 'marry', 'photo', 'reason', 'papa', 'get', 'really', 'twitchy', 'well', 'year', 'canada', 'announce', 'open', 'strip', 'citizenship', 'deport', 'war', 'criminal', 'follow', 'recommendation', 'deschene', 'commission', 'dad', 'drive', 'toronto', 'develop', 'relationship', 'dad', 'tag', 'sometimes', 'bring', 'carton', 'cigarette', 'chocolate', 'bar', 'get', 'parent', 'say', 'would', 'best', 'put', 'unpleasantness', 'start', 'fresh', 'bother', 'get', 'legally', 'divorce', 'pretend', 'happen', 'move', 'okay', 'go', 'tell', 'story', 'bigamous', 'grandmother', 'current', 'time', 'mute', 'hear', 'years_ago', 'steal', 'elite', 'daily', 'story', 'happen', 'last', 'time', 'point', 'mom', 'dial', 'number', 'handed', 'phone', 'dad', 'man', 'answer', 'know', 'marion', 'child', 'well', 'wait', 'call', 'year', 'dad',

In [68]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [69]:
for coro in corpus:
    print(coro)

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 3), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 6), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 3), (61, 1), (62, 1), (63, 2), (64, 1), (65, 2), (66, 3), (67, 2), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 5), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 2), (92, 1), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 4), (101, 1), (102, 1), (103, 1), (104, 1), (105, 2), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1),

[(22, 2), (165, 2), (183, 1), (204, 1), (245, 1), (263, 1), (266, 1), (364, 1), (372, 1), (377, 1), (424, 1), (425, 2), (446, 1), (516, 1), (538, 1), (549, 1), (555, 1), (564, 1), (596, 1), (631, 1), (637, 1), (660, 1), (661, 6), (662, 1), (663, 1), (664, 1), (665, 4), (666, 1), (667, 2), (668, 2), (669, 1), (670, 1), (671, 1), (672, 3), (673, 1), (674, 1), (675, 1), (676, 1), (677, 1), (678, 1), (679, 1), (680, 1), (681, 1), (682, 2), (683, 1), (684, 1), (685, 1), (686, 1), (687, 1), (688, 1), (689, 1), (690, 1), (691, 1), (692, 1), (693, 1), (694, 1), (695, 1), (696, 1), (697, 1)]
[(15, 1), (76, 1), (91, 1), (100, 1), (138, 1), (139, 1), (145, 1), (146, 1), (178, 1), (182, 1), (199, 2), (204, 1), (233, 1), (274, 1), (303, 1), (304, 1), (364, 1), (377, 1), (393, 3), (438, 1), (468, 3), (469, 2), (516, 2), (520, 2), (554, 1), (570, 1), (602, 1), (626, 1), (630, 2), (656, 1), (677, 1), (698, 1), (699, 1), (700, 1), (701, 1), (702, 1), (703, 1), (704, 1), (705, 4), (706, 1), (707, 1), (7

[(1, 1), (21, 10), (26, 1), (64, 1), (65, 1), (83, 1), (87, 1), (96, 1), (101, 1), (111, 1), (133, 1), (154, 1), (193, 1), (208, 1), (216, 1), (226, 1), (271, 2), (303, 1), (343, 1), (349, 1), (355, 1), (361, 2), (394, 1), (408, 2), (410, 1), (424, 1), (425, 1), (434, 1), (464, 3), (468, 4), (470, 1), (479, 1), (484, 4), (499, 1), (511, 1), (512, 1), (561, 1), (575, 1), (607, 1), (620, 1), (621, 1), (623, 1), (630, 1), (763, 1), (770, 1), (808, 1), (1055, 1), (1078, 1), (1099, 1), (1100, 4), (1101, 1), (1102, 1), (1103, 1), (1104, 1), (1105, 1), (1106, 1), (1107, 4), (1108, 1), (1109, 1), (1110, 1), (1111, 1), (1112, 3), (1113, 1), (1114, 1), (1115, 1), (1116, 1), (1117, 5), (1118, 1), (1119, 1), (1120, 1), (1121, 1), (1122, 1), (1123, 1), (1124, 1), (1125, 1), (1126, 1), (1127, 1), (1128, 2), (1129, 1), (1130, 1), (1131, 2), (1132, 1), (1133, 1), (1134, 1), (1135, 1), (1136, 1), (1137, 1), (1138, 1), (1139, 2), (1140, 1), (1141, 1), (1142, 1), (1143, 3), (1144, 1), (1145, 1), (1146, 1

[(119, 1), (174, 1), (190, 2), (193, 1), (244, 1), (261, 1), (291, 1), (318, 2), (355, 1), (377, 1), (394, 1), (500, 1), (513, 1), (516, 1), (545, 1), (564, 1), (569, 1), (578, 1), (587, 1), (590, 1), (621, 1), (644, 1), (668, 1), (700, 1), (721, 1), (729, 1), (764, 1), (768, 1), (773, 1), (777, 1), (779, 1), (844, 1), (859, 1), (895, 1), (930, 1), (983, 1), (996, 1), (1004, 2), (1051, 1), (1060, 1), (1102, 1), (1144, 1), (1207, 1), (1228, 1), (1281, 1), (1331, 1), (1348, 1), (1443, 2), (1485, 1), (1486, 1), (1487, 1), (1488, 1), (1489, 1), (1490, 1), (1491, 1), (1492, 1), (1493, 1), (1494, 2), (1495, 1), (1496, 1), (1497, 1), (1498, 1), (1499, 2), (1500, 1), (1501, 2), (1502, 2), (1503, 1), (1504, 1), (1505, 1), (1506, 1), (1507, 1), (1508, 1), (1509, 1), (1510, 1), (1511, 1), (1512, 1), (1513, 7), (1514, 2), (1515, 1), (1516, 1)]
[(64, 1), (148, 1), (174, 1), (211, 1), (268, 1), (279, 1), (578, 1), (890, 1), (1385, 1), (1517, 1), (1518, 1), (1519, 1)]
[(0, 2), (1, 1), (21, 12), (76, 

[(15, 1), (21, 2), (27, 1), (33, 1), (64, 1), (94, 1), (100, 1), (109, 1), (138, 1), (140, 1), (143, 1), (145, 1), (146, 1), (163, 2), (166, 2), (174, 2), (182, 1), (184, 2), (202, 2), (208, 1), (230, 1), (247, 1), (268, 1), (279, 1), (281, 1), (313, 1), (326, 1), (351, 1), (377, 1), (382, 1), (393, 1), (424, 1), (463, 1), (469, 1), (500, 2), (516, 1), (529, 1), (555, 1), (564, 2), (576, 1), (577, 1), (590, 1), (597, 1), (612, 2), (620, 1), (671, 1), (721, 1), (785, 1), (789, 2), (825, 1), (984, 1), (1011, 1), (1119, 1), (1140, 2), (1153, 1), (1188, 1), (1201, 3), (1227, 1), (1297, 2), (1332, 1), (1401, 1), (1432, 1), (1433, 1), (1470, 1), (1497, 1), (1524, 1), (1627, 1), (1848, 1), (1862, 1), (1863, 1), (1864, 1), (1865, 1), (1866, 1), (1867, 1), (1868, 1), (1869, 1), (1870, 1), (1871, 1), (1872, 1), (1873, 1), (1874, 1), (1875, 1), (1876, 1), (1877, 1), (1878, 1), (1879, 1), (1880, 1), (1881, 1), (1882, 1), (1883, 1), (1884, 1), (1885, 1), (1886, 1), (1887, 2), (1888, 1), (1889, 1), 

...

[(3, 3), (27, 1), (62, 1), (63, 1), (99, 2), (100, 1), (109, 11), (123, 1), (124, 1), (138, 3), (154, 1), (155, 3), (165, 1), (169, 2), (171, 2), (176, 1), (182, 1), (188, 1), (193, 2), (197, 1), (204, 1), (211, 1), (216, 1), (217, 1), (226, 1), (228, 1), (234, 3), (242, 1), (247, 1), (251, 6), (254, 1), (266, 1), (268, 10), (281, 1), (327, 1), (343, 1), (374, 1), (377, 5), (378, 6), (477, 2), (484, 1), (495, 1), (506, 1), (511, 1), (539, 1), (550, 1), (564, 1), (568, 1), (569, 1), (570, 1), (575, 1), (578, 3), (602, 1), (609, 1), (612, 2), (630, 1), (642, 1), (663, 9), (667, 1), (670, 3), (708, 1), (713, 1), (727, 1), (736, 2), (773, 3), (779, 1), (789, 1), (793, 1), (800, 1), (816, 1), (820, 1), (855, 1), (887, 1), (891, 1), (904, 1), (906, 1), (909, 1), (914, 3), (916, 1), (926, 1), (942, 1), (951, 1), (955, 1), (975, 1), (989, 1), (992, 1), (1011, 1), (1012, 1), (1013, 1), (1030, 2), (1039, 1), (1059, 2), (1078, 2), (1085, 1), (1119, 2), (1140, 1), (1150, 1), (1174, 1), (1176, 1), 

[(0, 1), (1, 3), (7, 5), (10, 1), (15, 1), (30, 1), (40, 1), (55, 5), (60, 1), (62, 4), (63, 1), (64, 8), (65, 1), (73, 1), (81, 1), (99, 3), (100, 3), (106, 2), (113, 1), (117, 2), (119, 3), (127, 1), (130, 1), (138, 1), (146, 3), (148, 4), (165, 2), (175, 1), (178, 1), (182, 3), (183, 1), (184, 3), (191, 1), (193, 3), (197, 1), (199, 4), (205, 1), (208, 1), (216, 1), (222, 1), (223, 2), (229, 2), (230, 4), (240, 1), (244, 2), (247, 2), (248, 3), (254, 1), (261, 1), (265, 1), (266, 3), (268, 3), (278, 1), (279, 1), (281, 5), (286, 1), (287, 3), (291, 4), (298, 2), (300, 5), (301, 1), (304, 1), (317, 1), (324, 1), (326, 3), (336, 1), (343, 3), (344, 2), (347, 3), (349, 1), (351, 1), (353, 1), (364, 4), (365, 1), (368, 1), (374, 1), (375, 1), (377, 3), (378, 1), (382, 4), (387, 1), (392, 3), (394, 1), (399, 1), (406, 1), (410, 4), (413, 2), (424, 7), (428, 2), (461, 5), (463, 1), (464, 2), (465, 1), (475, 1), (479, 2), (483, 2), (501, 2), (506, 1), (509, 1), (538, 1), (543, 2), (546, 4)

[(15, 1), (21, 2), (24, 7), (30, 1), (35, 1), (37, 2), (62, 1), (63, 3), (65, 3), (67, 1), (75, 1), (76, 1), (77, 1), (96, 1), (99, 2), (100, 6), (106, 3), (111, 1), (115, 1), (117, 3), (121, 2), (125, 1), (129, 1), (139, 9), (142, 1), (146, 3), (148, 1), (159, 1), (165, 1), (166, 1), (172, 3), (174, 2), (177, 3), (178, 1), (182, 2), (183, 5), (184, 7), (191, 1), (195, 3), (197, 1), (199, 2), (204, 1), (205, 1), (208, 1), (216, 2), (227, 2), (230, 1), (234, 1), (241, 2), (249, 2), (250, 2), (254, 3), (260, 2), (266, 3), (271, 1), (281, 4), (298, 5), (303, 1), (305, 2), (311, 2), (325, 1), (326, 1), (343, 1), (349, 2), (355, 1), (359, 1), (376, 1), (377, 1), (382, 1), (394, 1), (396, 1), (399, 1), (401, 1), (407, 2), (410, 1), (413, 1), (424, 2), (425, 1), (433, 1), (435, 1), (447, 1), (457, 1), (463, 4), (464, 3), (465, 3), (468, 5), (474, 1), (475, 1), (477, 1), (489, 3), (495, 1), (501, 3), (503, 1), (520, 2), (527, 1), (543, 1), (549, 2), (556, 3), (568, 1), (569, 1), (570, 1), (575

[(7, 1), (30, 2), (43, 1), (49, 1), (60, 2), (61, 10), (62, 1), (63, 1), (64, 2), (65, 2), (67, 6), (83, 1), (86, 2), (89, 1), (90, 3), (96, 2), (99, 2), (100, 3), (101, 1), (105, 2), (115, 1), (117, 3), (119, 2), (133, 1), (134, 1), (147, 1), (148, 1), (154, 8), (159, 1), (163, 3), (165, 3), (175, 1), (190, 3), (193, 3), (199, 3), (205, 1), (208, 1), (222, 1), (234, 1), (240, 2), (245, 1), (247, 2), (254, 1), (260, 1), (266, 3), (268, 1), (269, 1), (281, 2), (287, 1), (298, 1), (303, 1), (313, 1), (315, 1), (324, 1), (336, 3), (339, 1), (349, 1), (351, 2), (364, 1), (369, 2), (371, 3), (377, 2), (378, 2), (399, 1), (402, 1), (404, 1), (416, 1), (417, 1), (424, 4), (425, 1), (434, 1), (447, 1), (448, 1), (451, 2), (453, 1), (457, 1), (463, 1), (477, 2), (483, 1), (487, 1), (488, 1), (516, 1), (525, 5), (529, 1), (539, 2), (545, 2), (556, 1), (565, 1), (566, 2), (569, 4), (570, 1), (578, 3), (583, 1), (585, 1), (587, 1), (596, 1), (599, 1), (602, 4), (621, 2), (626, 3), (630, 1), (634, 

[(5, 3), (7, 3), (21, 1), (26, 1), (33, 1), (35, 1), (52, 1), (62, 4), (63, 2), (64, 7), (65, 1), (72, 2), (77, 1), (80, 1), (96, 2), (100, 1), (106, 2), (114, 1), (117, 3), (123, 1), (125, 1), (131, 1), (135, 1), (142, 2), (146, 4), (148, 3), (154, 1), (163, 3), (165, 1), (166, 2), (172, 2), (174, 1), (175, 2), (182, 5), (183, 1), (195, 1), (237, 3), (241, 2), (254, 1), (255, 3), (260, 1), (271, 5), (274, 1), (281, 2), (298, 1), (299, 1), (303, 1), (307, 1), (313, 1), (324, 5), (333, 1), (339, 3), (343, 3), (349, 2), (351, 1), (364, 1), (369, 1), (374, 3), (377, 2), (382, 1), (387, 2), (406, 2), (410, 2), (442, 3), (461, 5), (468, 2), (470, 2), (471, 1), (479, 3), (483, 4), (509, 1), (548, 1), (549, 1), (554, 1), (559, 3), (565, 1), (583, 4), (590, 1), (593, 2), (596, 2), (597, 2), (599, 2), (612, 5), (628, 1), (639, 2), (643, 2), (652, 1), (662, 3), (676, 1), (718, 1), (726, 1), (730, 1), (752, 1), (756, 1), (766, 2), (781, 3), (789, 2), (809, 1), (818, 1), (821, 2), (834, 1), (841, 

In [70]:
THIS_FOLDER = os.getcwd()
carpeta_mallet = "mallet-2.0.8"
bin_c = "bin"
archivo = "mallet"
camino = os.path.join(THIS_FOLDER, carpeta_mallet)
mallet_path_1 = os.path.join(camino, bin_c)
mallet_path = os.path.join(mallet_path_1,archivo)


os.environ['MALLET_HOME'] = camino

In [71]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [72]:
inicio = 2
limite = 40
pasos = 1

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=inicio, limit=limite, step=pasos)

In [ ]:
# Show graph
limit=limite
start=inicio
step=pasos;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[12]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))